In [ ]:
import random, pandas as pd, seaborn as sns, warnings, numpy as np, ast
from tqdm.notebook import tqdm as tqdm_notebook
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

from sklearn.model_selection import cross_validate

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

tf.config.run_functions_eagerly(True)

sns.set_theme(style="whitegrid")
warnings.filterwarnings("ignore")

# Set a random seed
random_seed = 42
random.seed(random_seed)

In [ ]:
path_database = "temp_gpt_4_1_nano_vecs.csv"

In [ ]:
database = pd.read_csv(path_database)
display(database.head(3))
database.shape

### Preparando setup de testes

In [ ]:
database.columns

In [ ]:
emb_cols = [c for c in database.columns if "embeddings" in c]
vec_names = [c.replace("embeddings_", "") for c in emb_cols]

# Convertendo strings de volta para listas
for ec in emb_cols:
    print(f"Convertendo {ec} para lista...")
    database[ec] = [ast.literal_eval(x) for x in database[ec]]

In [ ]:
i = 1
PART = {}
for c in database["company"].drop_duplicates():
    PART[c] = {}
    for v in vec_names:
        database_ = database[database["company"] == c]

        PART[c][v] = {
            "X": list(database_[f"embeddings_{v}"]),
            "y": list(database_["service_class"])
        }

    print(f"EMPRESA {i}: {c} - {vec_names} - {database_.shape}")
    i += 1

In [ ]:
def create_save_plot(c, df_model_metrics, save_file):
    fig = plt.figure(figsize=(8, 10))
    # fig.suptitle(c.capitalize(), fontsize=16)

    # Grade: 2 linhas, 2 colunas
    gs = GridSpec(2, 2, height_ratios=[2, 1])

    ax1 = fig.add_subplot(gs[0, :])
    ax2 = fig.add_subplot(gs[1, 0])
    ax3 = fig.add_subplot(gs[1, 1])

    # === Paletas modernas ===
    palette_main = [
        "#6366f1", "#10b981", "#f59e0b", "#ef4444", "#3b82f6",
        "#8b5cf6", "#ec4899", "#14b8a6", "#f97316", "#0ea5e9"
    ]

    palette_vec = [
        "#7dd3fc", "#93c5fd", "#c4b5fd", "#a5f3fc"
    ]

    palette_model = [
        "#4ade80", "#facc15", "#fb7185", "#60a5fa"
    ]

    # === Gráfico 1: Desempenho Geral ===
    sns.barplot(
        ax=ax1,
        data=df_model_metrics,
        x="model_name", y="test_f1_macro", hue="vec_name",
        errorbar="sd", palette=palette_main, alpha=.85
    )
    ax1.set_title("Overall performance")
    ax1.set_ylabel("F1 Macro")
    ax1.set_xlabel("")
    ax1.legend(title="", loc="upper right")

    # === Gráfico 2: Representação Vetorial ===
    sns.barplot(
        ax=ax2,
        data=df_model_metrics,
        x="vec_name", y="test_f1_macro",
        errorbar="sd", palette=palette_vec, alpha=.85
    )
    ax2.set_title("Vector representation")
    ax2.set_ylabel("")
    ax2.set_xlabel("")
    ax2.legend().remove()

    # === Gráfico 3: Modelo ===
    sns.barplot(
        ax=ax3,
        data=df_model_metrics,
        x="model_name", y="test_f1_macro",
        errorbar="sd", palette=palette_model, alpha=.85
    )
    ax3.set_title("Model")
    ax3.set_ylabel("")
    ax3.set_xlabel("")
    ax3.legend().remove()

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.savefig(save_file, dpi=300, bbox_inches="tight")

# TRADICIONAL

In [ ]:
models = {
    "RF": RandomForestClassifier(
        n_estimators=1000,                   # Mais árvores para estabilidade estatística e menor variância
        max_depth=100,                       # Maior profundidade para capturar padrões complexos
        max_features='log2',                 # Reduz ainda mais overfitting em dados textuais de alta dimensionalidade
        min_samples_split=10,                # Evita splits prematuros em amostras pequenas
        min_samples_leaf=4,                  # Evita folhas muito pequenas
        bootstrap=True,                      # Mantém o bootstrap ativado para diversidade
        class_weight='balanced_subsample',   # Compensa classes desbalanceadas durante bootstrap
        n_jobs=-1,
        random_state=42
    ),

    "SVM": SVC(
        C=1.0,                               # Regularização moderada. Pode testar 0.1, 1.0, 10 para tuning
        kernel='rbf',                        # Kernel RBF captura padrões não lineares (recomendado para embeddings densos como BERT)
        gamma='scale',                       # Define gamma automaticamente como 1 / n_features, robusto para começar
        probability=True,                    # Permite predict_proba(), útil para ROC AUC ou thresholds probabilísticos
        class_weight='balanced',             # Compensa classes desbalanceadas automaticamente
        max_iter=10000,                      # Aumentado para evitar não convergir em datasets médios
        random_state=42                      # Reprodutibilidade
    ),

    "MLP": MLPClassifier(
        hidden_layer_sizes=(1024, 512, 128), # Arquitetura mais profunda, útil para capturar interações não-lineares complexas
        activation='relu',
        solver='adam',
        alpha=1e-3,                          # Regularização mais forte (mais adequada para vetores textuais esparsos)
        learning_rate='adaptive',
        early_stopping=True,
        validation_fraction=0.1,             # Mantém 10% para validação interna no early stopping
        max_iter=500,                        # Permite mais épocas (mas com early stopping)
        batch_size=32,                       # Reduzido para maior precisão gradiente em vetores pequenos
        random_state=42
    )
}

In [ ]:
model_metrics_all = []
for c in tqdm_notebook(PART.keys()):
    model_metrics = []
    for v in PART[c].keys():
        for m in models.keys():
            cv_results = cross_validate(
                models[m], 
                PART[c][v]["X"], 
                PART[c][v]["y"], 
                cv=2,
                scoring=('accuracy', 'precision', 'recall', 'f1_macro'),
                return_train_score=True
            )
            
            cvr = pd.DataFrame(cv_results)
            cvr["company"] = c
            cvr["model_name"] = m
            cvr["vec_name"] = v

            mean = round(cvr['test_f1_macro'].mean(), 2)
            std = round(cvr['test_f1_macro'].std(), 2)
            print(f"EMPRESA: {c}\tVEC: {v}\tMODEL: {m}\tR: {mean}\t+- {std}")

            model_metrics.append(cvr)

    save_file = f"./plot_results/{c}.png".lower()
    create_save_plot(c, model_metrics, save_file)

    model_metrics_all += model_metrics

In [ ]:
df_model_metrics_all = pd.concat(model_metrics_all)
df_model_metrics_all.to_excel("reclamacoes_results.xlsx")

# CNN

In [ ]:
def create_plot_cnn(df, save_file, n_epochs):
    # Limpa qualquer figura anterior
    plt.clf()  # limpa o conteúdo atual da figura
    plt.close('all')  # fecha figuras abertas (opcional para liberar memória)

    # Configurações do seaborn
    sns.set(rc={'figure.figsize': (12, 9)})
    sns.set_style("ticks", {'axes.grid': True, 'grid.color': '.95'})
    sns.set_context("notebook", font_scale=1.2, rc={"lines.linewidth": 2.5})

    # Cria a figura e o eixo explicitamente
    fig, ax = plt.subplots()

    # Cria o gráfico
    sns.lineplot(
        x="Epochs",
        y="Variation",
        hue="Metrics",
        data=df,
        ax=ax  # usa o eixo explicitamente para evitar global state
    )

    # Configura o gráfico
    ax.legend(loc='center right', bbox_to_anchor=(1.2, 0.5))
    ax.set(xlim=(1, n_epochs))

    # Salva o gráfico
    plt.savefig(save_file, dpi=300, bbox_inches="tight")

    # Fecha a figura para garantir que nada fique em memória
    plt.close(fig)

In [ ]:
# =============================
# Configurações iniciais
# =============================
part_test = 0.1
part_valid = 0.1
n_epocas = 100
repeticao_model = 2
semente_model = True  # False: pesos aleatórios, True: sementes variando, int: fixa a semente

for c in tqdm_notebook(PART.keys()):
    model_metrics = []
    for v in PART[c].keys():
        # =============================
        # Carregamento e pré-processamento
        # =============================
        X_all = np.array(PART[c][v]["X"])
        Y_all = np.array(PART[c][v]["y"])

        lb = LabelBinarizer()
        Y_all_encoded = lb.fit_transform(Y_all)
        if Y_all_encoded.shape[1] == 1:  # caso binário
            Y_all_encoded = np.hstack((1 - Y_all_encoded, Y_all_encoded))

        X_train_all, X_test, y_train_all, y_test = train_test_split(X_all, Y_all_encoded, test_size=part_test, random_state=13)
        X_train, X_valid, y_train, y_valid = train_test_split(X_train_all, y_train_all, test_size=part_valid, random_state=13)

        # =============================
        # Inicialização de métricas
        # =============================
        histories = []
        predictions = []

        # =============================
        # Treinamento
        # =============================
        cv_results = {
            'fit_time': [],
            'score_time': [],
            'test_accuracy': [],
            'train_accuracy': [],
            'test_precision': [],
            'train_precision': [],
            'test_recall': [],
            'train_recall': [],
            'test_f1_macro': [],
            'train_f1_macro': []
        }

        for i in range(repeticao_model):
            # Define semente
            seed = i if semente_model is True else (semente_model if isinstance(semente_model, int) else None)
            if seed is not None:
                np.random.seed(seed)
                tf.random.set_seed(seed)
                print(f"Semente = {seed}")

            # Otimizador
            sgd = SGD(learning_rate=0.01, nesterov=True)

            # Modelo
            model = Sequential([
                layers.Input(shape=(X_train.shape[1], X_train.shape[2])),  # (seq_len, embedding_dim)

                layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'),
                layers.MaxPooling1D(pool_size=2),

                layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
                layers.MaxPooling1D(pool_size=2),

                layers.GlobalMaxPooling1D(),

                layers.Dense(128, activation='relu'),
                layers.Dropout(0.3),

                layers.Dense(Y_all_encoded.shape[1], activation='softmax')
            ])

            model.compile(
                loss='categorical_crossentropy',
                optimizer=sgd,
                metrics=['accuracy']
            )

            # ================================
            # Medição de tempo de treinamento
            # ================================
            start_fit = time.time()
            history = model.fit(
                X_train, y_train,
                validation_data=(X_valid, y_valid),
                epochs=n_epocas,
                verbose=1
            )
            histories.append(history.history)
            
            end_fit = time.time()
            fit_duration = end_fit - start_fit
            cv_results['fit_time'].append(fit_duration)

            # ================================
            # Medição de tempo de predição
            # ================================
            start_score = time.time()
            y_pred_test = np.argmax(model.predict(X_test, verbose=0), axis=1)
            y_pred_train = np.argmax(model.predict(X_train_all, verbose=0), axis=1)
            end_score = time.time()
            score_duration = end_score - start_score
            cv_results['score_time'].append(score_duration)

            # Decodificar labels para índices
            y_true_test = np.argmax(y_test, axis=1)
            y_true_train = np.argmax(y_train_all, axis=1)

            # ================================
            # Cálculo de métricas
            # ================================
            cv_results['test_accuracy'].append(accuracy_score(y_true_test, y_pred_test))
            cv_results['train_accuracy'].append(accuracy_score(y_true_train, y_pred_train))

            cv_results['test_precision'].append(precision_score(y_true_test, y_pred_test, average='macro', zero_division=0))
            cv_results['train_precision'].append(precision_score(y_true_train, y_pred_train, average='macro', zero_division=0))

            cv_results['test_recall'].append(recall_score(y_true_test, y_pred_test, average='macro', zero_division=0))
            cv_results['train_recall'].append(recall_score(y_true_train, y_pred_train, average='macro', zero_division=0))

            cv_results['test_f1_macro'].append(f1_score(y_true_test, y_pred_test, average='macro', zero_division=0))
            cv_results['train_f1_macro'].append(f1_score(y_true_train, y_pred_train, average='macro', zero_division=0))

        cvr = pd.DataFrame(cv_results)
        cvr["company"] = c
        cvr["model_name"] = m
        cvr["vec_name"] = v

        model_metrics.append(cvr)

        # =============================
        # Organização das métricas
        # =============================
        # Concatena todas as métricas
        all_metrics = {
            'loss': [],
            'val_loss': [],
            'accuracy': [],
            'val_accuracy': []
        }

        for h in histories:
            for k in all_metrics.keys():
                all_metrics[k].extend(h[k])

        # Cria DataFrame
        metrica_labels = ['Train loss'] * n_epocas * repeticao_model + \
                         ['Val loss'] * n_epocas * repeticao_model + \
                         ['Train Acc'] * n_epocas * repeticao_model + \
                         ['Val Acc'] * n_epocas * repeticao_model

        epocas = np.tile(np.arange(1, n_epocas + 1), 4 * repeticao_model)
        dados = all_metrics['loss'] + all_metrics['val_loss'] + all_metrics['accuracy'] + all_metrics['val_accuracy']

        df = pd.DataFrame({
            'Epochs': epocas,
            'Metrics': metrica_labels,
            'Variation': dados
        })

        # =============================
        # Plot
        # =============================
        save_file = f"/plot_results/cnn/{c}_{v}.png".lower()
        create_plot_cnn(df, save_file, n_epocas)

    model_metrics_all += model_metrics

In [ ]:
df_model_metrics_all = pd.concat(model_metrics_all)
df_model_metrics_all.to_excel("reclamacoes_results.xlsx")